In [1]:
%run setup.ipynb

# IR alleles - supplementary table
per sample allele counts for known IR loci - kdr, ace1, gste2 
- GSTE and ace1 variants are only present in the raw data - need to use this and provide caveats
- this could be merged with the main meta data table?
- are there any more loci we could include?

In [2]:
df_meta = pd.read_csv('../meta/ag1000g/phase2_samples.meta.txt', sep='\t')
df_meta.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0051-C,Twifo_Praso__C1,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,99910300,33.33,ERS311871,5.60858,-1.54926
3,AA0052-C,Twifo_Praso__F2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,100899280,32.29,ERS311879,5.60858,-1.54926
4,AA0053-C,Twifo_Praso__A4,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,104791234,34.14,ERS311887,5.60858,-1.54926


In [3]:
callset_fn = '../phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1/'
callset = zarr.open_group(callset_fn) 

## kdr - 995F/S/Y

In [4]:
genk = allel.GenotypeDaskArray(callset['2L']['calldata/genotype'])
posk = allel.SortedIndex(callset['2L']['variants']['POS'][:])

In [5]:
#kdr - 995F and S
F = genk[posk.locate_range(2422652, 2422652)]
Y = genk[posk.locate_range(2429745, 2429745)]
S = genk[posk.locate_range(2422651, 2422651)]

In [6]:
kf = []
ky = []
ks = []
for f, y, s in zip(np.asarray(F)[0], np.asarray(Y)[0], np.asarray(S)[0]):
    kf.append((f[0] + f[1]))
    ky.append((y[0] + y[1]))
    ks.append((s[0] + s[1]))

## ace-1 - G119S

In [7]:
gena = allel.GenotypeDaskArray(callset['2R']['calldata/GT'])
posa = allel.SortedIndex(callset['2R']['variants']['POS'][:])

In [8]:
A = gena[posa.locate_range(3492074, 3492074)]

In [9]:
a1 = []
for a in np.asarray(A)[0]:
    a1.append((a[0] + a[1]))

## GSTE2 - I114T

In [10]:
geng = allel.GenotypeDaskArray(callset['3R']['calldata/genotype'])
posg = allel.SortedIndex(callset['3R']['variants']['POS'][:])

In [11]:
G = geng[posg.locate_range(28598166, 28598166)]

In [12]:
gi = []
for g in np.asarray(G)[0]:
    gi.append((g[0] + g[1]))

## Supplementary table - per sample allele count - separate table

In [13]:
sp = df_meta['m_s']
sp.value_counts()

S      654
M      283
M/S      1
Name: m_s, dtype: int64

In [14]:
sp.replace(['S', 'M', 'M/S'], ['gambiae', 'coluzzii', 'hybrid'], inplace=True)
sp.value_counts()

gambiae     654
coluzzii    283
hybrid        1
Name: m_s, dtype: int64

In [15]:
sample_df = pandas.DataFrame()
sample_df['ox_code'] = df_meta['ox_code']
sample_df['population'] = df_meta['population']
sample_df['country'] = df_meta['country']
sample_df['location'] = df_meta['location']
sample_df['site'] = df_meta['site']
sample_df['year'] = df_meta['year']
sample_df['species'] = sp
sample_df['latitude'] = df_meta['latitude']
sample_df['longitude'] = df_meta['longitude']
sample_df['vgsc-L995F'] = kf
sample_df['vgsc-N1570Y'] = ky
sample_df['vgsc-L995S'] = ks
sample_df['ace1-G119S'] = a1
sample_df['gste2-I114T'] = gi
sample_df.head()

,ox_code,population,country,location,site,year,species,latitude,longitude,vgsc-L995F,vgsc-N1570Y,vgsc-L995S,ace1-G119S,gste2-I114T
0,AA0040-C,GHcol,Ghana,Twifo Praso,Twifo Praso,2012,coluzzii,5.60858,-1.54926,2,0,0,0,0
1,AA0041-C,GHcol,Ghana,Twifo Praso,Twifo Praso,2012,coluzzii,5.60858,-1.54926,2,0,0,0,2
2,AA0051-C,GHcol,Ghana,Twifo Praso,Twifo Praso,2012,coluzzii,5.60858,-1.54926,2,0,0,0,0
3,AA0052-C,GHcol,Ghana,Twifo Praso,Twifo Praso,2012,coluzzii,5.60858,-1.54926,1,0,0,0,1
4,AA0053-C,GHcol,Ghana,Twifo Praso,Twifo Praso,2012,coluzzii,5.60858,-1.54926,1,0,0,0,0


In [16]:
sample_df.to_csv('../tables/sample_IR_counts.csv', sep='\t', index=False)

## Supplementary table - per sample allele count - combined with meta table

In [17]:
to_join_df = sample_df[['ox_code', 'vgsc-L995F', 'vgsc-N1570Y', 'vgsc-L995S', 'ace1-G119S', 'gste2-I114T']]
to_join_df.head()

,ox_code,vgsc-L995F,vgsc-N1570Y,vgsc-L995S,ace1-G119S,gste2-I114T
0,AA0040-C,2,0,0,0,0
1,AA0041-C,2,0,0,0,2
2,AA0051-C,2,0,0,0,0
3,AA0052-C,1,0,0,0,1
4,AA0053-C,1,0,0,0,0


In [18]:
sample_combined_df = df_meta.merge(to_join_df, on='ox_code')
sample_combined_df.to_csv('../tables/sample_IR_counts_meta_combined.csv', sep='\t', index=False)

In [19]:
sample_combined_df.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,...,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude,vgsc-L995F,vgsc-N1570Y,vgsc-L995S,ace1-G119S,gste2-I114T
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,coluzzii,...,95033368,30.99,ERS311878,5.60858,-1.54926,2,0,0,0,0
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,coluzzii,...,95843804,31.70,ERS311886,5.60858,-1.54926,2,0,0,0,2
2,AA0051-C,Twifo_Praso__C1,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,coluzzii,...,99910300,33.33,ERS311871,5.60858,-1.54926,2,0,0,0,0
3,AA0052-C,Twifo_Praso__F2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,coluzzii,...,100899280,32.29,ERS311879,5.60858,-1.54926,1,0,0,0,1
4,AA0053-C,Twifo_Praso__A4,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,coluzzii,...,104791234,34.14,ERS311887,5.60858,-1.54926,1,0,0,0,0


## per population allele frequencies

In [20]:
pop_df = sample_df[['population', 'vgsc-L995F', 'vgsc-N1570Y', 'vgsc-L995S', 'ace1-G119S', 'gste2-I114T']].groupby(['population']).sum()
pop_df.reset_index(inplace=True)
pop_df.set_index('population', inplace=True)
pop_df.head()

,vgsc-L995F,vgsc-N1570Y,vgsc-L995S,ace1-G119S,gste2-I114T
population,,,,,
AOcol,131,0,0,0,1
BFcol,145,31,0,14,38
BFgam,167,47,0,18,51
CIcol,130,0,0,62,20
CMgam,312,34,93,0,5


In [21]:
for p in phase2_ar1.pop_ids:
    pop_df = pop_df.apply(lambda x: (x / (len(df_meta[df_meta.population == p]) * 2) * 100) if x.name == p else x, axis=1).round(decimals=2)
pop_df

,vgsc-L995F,vgsc-N1570Y,vgsc-L995S,ace1-G119S,gste2-I114T
population,,,,,
AOcol,83.97,0.00,0.00,0.00,0.64
BFcol,96.67,20.67,0.00,9.33,25.33
BFgam,90.76,25.54,0.00,9.78,27.72
CIcol,91.55,0.00,0.00,43.66,14.08
CMgam,52.53,5.72,15.66,0.00,0.84
FRgam,0.00,0.00,0.00,0.00,0.00
GAgam,32.61,0.00,67.39,0.00,28.99
GHcol,88.18,2.73,0.00,18.18,34.55
GHgam,70.83,4.17,0.00,12.50,33.33


In [23]:
pop_df.to_csv('../tables/population_IR_frequency.csv', sep='\t', index=True)